In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# URLs de los archivos de datos
url1 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_1%20.csv"
url2 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_2.csv"
url3 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_3.csv"
url4 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_4.csv"

# Cargar los datos 
print("Cargando datos de las tiendas...")
tienda1 = pd.read_csv(url1)
tienda2 = pd.read_csv(url2)
tienda3 = pd.read_csv(url3)
tienda4 = pd.read_csv(url4)

# Agregar identificador de tienda para facilitar el análisis
tienda1['Tienda'] = 'Tienda 1'
tienda2['Tienda'] = 'Tienda 2'
tienda3['Tienda'] = 'Tienda 3'
tienda4['Tienda'] = 'Tienda 4'

# Combinar todos los datos para análisis comparativos
todas_tiendas = pd.concat([tienda1, tienda2, tienda3, tienda4])

print("Análisis exploratorio inicial")
print("-" * 50)

# Explorar la estructura de los datos
print("\nEstructura del conjunto de datos:")
print(f"Columnas disponibles: {tienda1.columns.tolist()}")
print(f"Tipos de datos:\n{tienda1.dtypes}")

# Ver las primeras filas para entender el contenido
print("\nPrimeras filas de la tienda 1:")
print(tienda1.head())

# Ver información general de cada tienda
print("\nResumen de cada tienda:")
for i, tienda in enumerate([tienda1, tienda2, tienda3, tienda4], 1):
    print(f"\nTienda {i}:")
    print(f"- Número de registros: {tienda.shape[0]}")
    print(f"- Número de columnas: {tienda.shape[1]}")
    print(f"- Valores nulos: {tienda.isnull().sum().sum()}")
    print(f"- Categorías de productos únicas: {tienda['Categoría del Producto'].nunique()}")
    print(f"- Productos únicos: {tienda['Producto'].nunique()}")

# 1. Análisis de facturación por tienda
print("\n1. ANÁLISIS DE FACTURACIÓN")
print("-" * 50)

# Calcular el ingreso total por tienda
ingresos_tiendas = []
for i, tienda in enumerate([tienda1, tienda2, tienda3, tienda4], 1):
    ingreso_total = tienda['Precio'].sum()
    ingresos_tiendas.append(ingreso_total)
    print(f"Tienda {i} - Ingreso total: ${ingreso_total:,.2f}")

# Crear un diccionario para facilitar el uso posterior
ingresos_dict = {
    'Tienda 1': ingresos_tiendas[0],
    'Tienda 2': ingresos_tiendas[1],
    'Tienda 3': ingresos_tiendas[2],
    'Tienda 4': ingresos_tiendas[3]
}

# Visualizar ingresos totales por tienda
plt.figure(figsize=(10, 6))
tiendas = ['Tienda 1', 'Tienda 2', 'Tienda 3', 'Tienda 4']
plt.bar(tiendas, ingresos_tiendas, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
plt.title('Ingresos Totales por Tienda', fontsize=16)
plt.xlabel('Tienda', fontsize=12)
plt.ylabel('Ingresos Totales ($)', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Agregar etiquetas de valor en las barras
for i, v in enumerate(ingresos_tiendas):
    plt.text(i, v + (max(ingresos_tiendas) * 0.02), f'${v:,.0f}', 
             ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig('ingresos_por_tienda.png')
plt.close()

# 2. Análisis de ventas por categoría
print("\n2. ANÁLISIS DE VENTAS POR CATEGORÍA")
print("-" * 50)

# Definir una función para analizar ventas por categoría
def analizar_ventas_por_categoria(tienda, nombre_tienda):
    ventas_por_categoria = tienda.groupby('Categoría del Producto')['Precio'].sum().sort_values(ascending=False)
    print(f"\n{nombre_tienda} - Ventas por categoría:")
    for categoria, venta in ventas_por_categoria.items():
        print(f"- {categoria}: ${venta:,.2f}")
    return ventas_por_categoria

# Analizar cada tienda
ventas_cat_t1 = analizar_ventas_por_categoria(tienda1, "Tienda 1")
ventas_cat_t2 = analizar_ventas_por_categoria(tienda2, "Tienda 2")
ventas_cat_t3 = analizar_ventas_por_categoria(tienda3, "Tienda 3")
ventas_cat_t4 = analizar_ventas_por_categoria(tienda4, "Tienda 4")

# Visualizar las categorías principales para cada tienda (top 3)
fig, axs = plt.subplots(2, 2, figsize=(15, 12))
axs = axs.flatten()

for i, (ventas, tienda_nombre) in enumerate(zip(
    [ventas_cat_t1, ventas_cat_t2, ventas_cat_t3, ventas_cat_t4], 
    ['Tienda 1', 'Tienda 2', 'Tienda 3', 'Tienda 4'])):
    
    # Ordenar y tomar las 3 principales categorías
    top_categorias = ventas.sort_values(ascending=False).head(3)
    
    # Crear gráfico
    axs[i].pie(top_categorias, labels=top_categorias.index, autopct='%1.1f%%', 
               startangle=90, shadow=True, 
               colors=plt.cm.tab10.colors[:len(top_categorias)])
    axs[i].set_title(f'Top 3 Categorías - {tienda_nombre}', fontsize=14)

plt.tight_layout()
plt.savefig('categorias_por_tienda.png')
plt.close()

# 3. Análisis de calificaciones promedio
print("\n3. ANÁLISIS DE CALIFICACIONES PROMEDIO")
print("-" * 50)

# Calcular calificación promedio por tienda
calificaciones = []
for i, tienda in enumerate([tienda1, tienda2, tienda3, tienda4], 1):
    cal_prom = tienda['Calificación'].mean()
    calificaciones.append(cal_prom)
    print(f"Tienda {i} - Calificación promedio: {cal_prom:.2f}/5")

# Visualizar calificaciones promedio
plt.figure(figsize=(10, 6))
tiendas = ['Tienda 1', 'Tienda 2', 'Tienda 3', 'Tienda 4']
bars = plt.bar(tiendas, calificaciones, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])

# Personalizar colores según calificación (verde=bueno, rojo=malo)
for i, bar in enumerate(bars):
    if calificaciones[i] >= 4:
        bar.set_color('#2ca02c')  
    elif calificaciones[i] < 3:
        bar.set_color('#d62728')  
    else:
        bar.set_color('#ff7f0e')  

plt.title('Calificación Promedio por Tienda', fontsize=16)
plt.xlabel('Tienda', fontsize=12)
plt.ylabel('Calificación Promedio (1-5)', fontsize=12)
plt.ylim(0, 5)  # Establecer límites del eje y
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Agregar etiquetas de valor en las barras
for i, v in enumerate(calificaciones):
    plt.text(i, v + 0.1, f'{v:.2f}', ha='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig('calificaciones_por_tienda.png')
plt.close()

# 4. Análisis de productos más y menos vendidos
print("\n4. ANÁLISIS DE PRODUCTOS MÁS Y MENOS VENDIDOS")
print("-" * 50)

# Definir función para analizar productos vendidos
def analizar_productos_vendidos(tienda, nombre_tienda):
    # Contar frecuencia de productos
    conteo_productos = tienda['Producto'].value_counts()
    
    # Productos más vendidos (top 5)
    mas_vendidos = conteo_productos.head(5)
    print(f"\n{nombre_tienda} - Productos más vendidos:")
    for producto, cantidad in mas_vendidos.items():
        print(f"- {producto}: {cantidad} unidades")
    
    # Productos menos vendidos (últimos 5)
    menos_vendidos = conteo_productos.tail(5)
    print(f"\n{nombre_tienda} - Productos menos vendidos:")
    for producto, cantidad in menos_vendidos.items():
        print(f"- {producto}: {cantidad} unidades")
    
    return mas_vendidos, menos_vendidos

# Analizar cada tienda
productos_t1 = analizar_productos_vendidos(tienda1, "Tienda 1")
productos_t2 = analizar_productos_vendidos(tienda2, "Tienda 2")
productos_t3 = analizar_productos_vendidos(tienda3, "Tienda 3")
productos_t4 = analizar_productos_vendidos(tienda4, "Tienda 4")

# 5. Análisis de costo de envío promedio
print("\n5. ANÁLISIS DE COSTO DE ENVÍO PROMEDIO")
print("-" * 50)

# Calcular costo de envío promedio por tienda
costos_envio = []
for i, tienda in enumerate([tienda1, tienda2, tienda3, tienda4], 1):
    costo_prom = tienda['Costo de envío'].mean()
    costos_envio.append(costo_prom)
    print(f"Tienda {i} - Costo de envío promedio: ${costo_prom:,.2f}")

# Visualizar costos de envío promedio
plt.figure(figsize=(10, 6))
tiendas = ['Tienda 1', 'Tienda 2', 'Tienda 3', 'Tienda 4']
plt.bar(tiendas, costos_envio, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
plt.title('Costo de Envío Promedio por Tienda', fontsize=16)
plt.xlabel('Tienda', fontsize=12)
plt.ylabel('Costo de Envío Promedio ($)', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Agregar etiquetas de valor en las barras
for i, v in enumerate(costos_envio):
    plt.text(i, v + (max(costos_envio) * 0.02), f'${v:,.2f}', 
             ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig('costos_envio_por_tienda.png')
plt.close()

# 6. Análisis de relación entre precio y calificación
print("\n6. ANÁLISIS ADICIONAL: RELACIÓN PRECIO-CALIFICACIÓN")
print("-" * 50)

# Crear un gráfico de dispersión para visualizar la relación
plt.figure(figsize=(12, 8))

# Definir colores para cada tienda
colores = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
tiendas_data = [tienda1, tienda2, tienda3, tienda4]
nombres_tiendas = ['Tienda 1', 'Tienda 2', 'Tienda 3', 'Tienda 4']

# Crear scatter plot para cada tienda
for i, (tienda, nombre, color) in enumerate(zip(tiendas_data, nombres_tiendas, colores)):
    plt.scatter(tienda['Precio'], tienda['Calificación'], 
                alpha=0.6, label=nombre, color=color, edgecolors='w', linewidth=0.5)

plt.title('Relación entre Precio y Calificación por Tienda', fontsize=16)
plt.xlabel('Precio del Producto ($)', fontsize=12)
plt.ylabel('Calificación (1-5)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend()
plt.tight_layout()
plt.savefig('precio_vs_calificacion.png')
plt.close()

# Resumen y recomendación final
print("\nRESUMEN Y RECOMENDACIÓN FINAL")
print("-" * 50)

# Crear un DataFrame para comparar todas las métricas
metricas_comparativas = pd.DataFrame({
    'Tienda': tiendas,
    'Ingresos Totales': ingresos_tiendas,
    'Calificación Promedio': calificaciones,
    'Costo de Envío Promedio': costos_envio
})

print("\nMétricas comparativas de todas las tiendas:")
print(metricas_comparativas)

# Crear una puntuación ponderada para cada tienda (mayor es mejor)
# Normalizar cada métrica para que esté en escala similar
max_ingresos = max(ingresos_tiendas)
max_envio = max(costos_envio)

metricas_comparativas['Puntuación Ingresos'] = metricas_comparativas['Ingresos Totales'] / max_ingresos
metricas_comparativas['Puntuación Calificación'] = metricas_comparativas['Calificación Promedio'] / 5
metricas_comparativas['Puntuación Envío'] = 1 - (metricas_comparativas['Costo de Envío Promedio'] / max_envio)  # Menor costo es mejor

# Calcular puntuación final (ponderada)
pesos = {
    'ingresos': 0.4,      # 40% importancia a ingresos
    'calificacion': 0.4,  # 40% importancia a calificaciones de clientes
    'envio': 0.2          # 20% importancia a costos de envío
}

metricas_comparativas['Puntuación Final'] = (
    metricas_comparativas['Puntuación Ingresos'] * pesos['ingresos'] +
    metricas_comparativas['Puntuación Calificación'] * pesos['calificacion'] +
    metricas_comparativas['Puntuación Envío'] * pesos['envio']
)

# Ordenar tiendas por puntuación final (de peor a mejor)
metricas_comparativas = metricas_comparativas.sort_values('Puntuación Final')

print("\nPuntuación final (mayor es mejor):")
for i, row in metricas_comparativas.iterrows():
    print(f"{row['Tienda']}: {row['Puntuación Final']:.3f}")

# Identificar la tienda menos eficiente 
tienda_menos_eficiente = metricas_comparativas.iloc[0]['Tienda']
print(f"\n=> La tienda menos eficiente es: {tienda_menos_eficiente}")

# Crear un informe detallado con la recomendación
print("\n" + "=" * 80)
print("INFORME FINAL Y RECOMENDACIÓN PARA EL SR. JUAN")
print("=" * 80)

print(f"""
Después de un análisis exhaustivo de las cuatro tiendas de la cadena Alura Store,
recomendamos que el Sr. Juan venda la {tienda_menos_eficiente} para iniciar
su nuevo emprendimiento. Esta recomendación se basa en los siguientes factores:

1. INGRESOS:
   La {tienda_menos_eficiente} generó ${metricas_comparativas.iloc[0]['Ingresos Totales']:,.2f} en ingresos,
   que es {'el menor' if metricas_comparativas.iloc[0]['Puntuación Ingresos'] < metricas_comparativas.iloc[1]['Puntuación Ingresos'] else 'uno de los menores'} entre todas las tiendas.
   Esto indica que la tienda tiene un potencial de crecimiento limitado en comparación
   con las demás tiendas de la cadena.

2. SATISFACCIÓN DEL CLIENTE:
   La calificación promedio de {tienda_menos_eficiente} es de {metricas_comparativas.iloc[0]['Calificación Promedio']:.2f}/5,
   {'la más baja' if metricas_comparativas.iloc[0]['Calificación Promedio'] < metricas_comparativas.iloc[1]['Calificación Promedio'] else 'una de las más bajas'} entre todas las tiendas.
   Esto sugiere que los clientes no están tan satisfechos con los productos 
   o el servicio ofrecido por esta tienda.

3. COSTOS DE ENVÍO:
   El costo de envío promedio de la {tienda_menos_eficiente} es de ${metricas_comparativas.iloc[0]['Costo de Envío Promedio']:,.2f},
   {'el más alto' if metricas_comparativas.iloc[0]['Puntuación Envío'] < metricas_comparativas.iloc[1]['Puntuación Envío'] else 'uno de los más altos'} entre todas las tiendas.
   Estos costos elevados afectan la rentabilidad general del negocio.

4. PRODUCTOS Y CATEGORÍAS:
   El análisis de productos más vendidos muestra que la {tienda_menos_eficiente} 
   no tiene un desempeño destacado en ninguna categoría específica que pudiera
   compensar sus debilidades en otras áreas.

Al vender la {tienda_menos_eficiente}, el Sr. Juan podrá liberar capital para su nuevo
emprendimiento mientras mantiene las tiendas más rentables y mejor evaluadas
de la cadena Alura Store. Esta decisión maximizará el valor total de su negocio
y le permitirá centrarse en mejorar las tiendas restantes o invertir en su nuevo proyecto.

Recomendación adicional: Antes de proceder con la venta, sería prudente realizar
un análisis más detallado de la ubicación geográfica y la competencia local
para confirmar que no hay factores externos temporales que estén afectando
el rendimiento de la {tienda_menos_eficiente}.
""")

# Crear un gráfico final de resumen con todas las métricas principales
print("\nCreando gráfico de resumen final...")

# Preparar datos para el gráfico de radar
categorias = ['Ingresos', 'Calificación', 'Eficiencia de Envío']
tiendas_radar = metricas_comparativas['Tienda'].tolist()

# Normalizar valores para el gráfico de radar (escala 0-1)
valores = np.zeros((len(tiendas_radar), len(categorias)))
valores[:, 0] = metricas_comparativas['Puntuación Ingresos']
valores[:, 1] = metricas_comparativas['Puntuación Calificación']
valores[:, 2] = metricas_comparativas['Puntuación Envío']

# Número de variables
N = len(categorias)

# Calcular ángulos para cada eje del gráfico de radar
angulos = [n / float(N) * 2 * np.pi for n in range(N)]
angulos += angulos[:1]  # Cerrar el círculo

# Crear figura
plt.figure(figsize=(10, 10))
ax = plt.subplot(111, polar=True)

# Dibujar líneas del gráfico
for i, tienda in enumerate(tiendas_radar):
    valores_tienda = valores[i].tolist()
    valores_tienda += valores_tienda[:1]  # Cerrar el círculo
    ax.plot(angulos, valores_tienda, linewidth=2, label=tienda, color=colores[i])
    ax.fill(angulos, valores_tienda, alpha=0.25, color=colores[i])

# Configurar etiquetas
plt.xticks(angulos[:-1], categorias)
ax.set_title('Comparación de Tiendas - Métricas Clave', fontsize=16)
plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))

plt.tight_layout()
plt.savefig('comparacion_radar.png')
plt.close()

print("\nAnálisis completado. Se han generado todos los gráficos y el informe final con la recomendación para el Sr. Juan.")

# Análisis adicionales que podrían ser útiles

# 1. Comparación de eficiencia por metros cuadrados.
print("\n=== ANÁLISIS ADICIONALES QUE PODRÍAN REALIZARSE ===")
print("-" * 50)
print("""
Para un análisis más completo, el Sr. Juan podría considerar recopilar y analizar:
1. Datos de metros cuadrados de cada tienda para calcular eficiencia por espacio
2. Costos operativos (alquiler, personal, servicios) para análisis de rentabilidad
3. Información de competencia local para entender el contexto de mercado
4. Tendencias temporales para identificar patrones estacionales
5. Datos demográficos de clientes para mejorar estrategias de marketing
""")

# 2. Análisis temporal (evolución de ventas a lo largo del tiempo)
# Convertir fechas a formato datetime para análisis cronológico
for tienda in [tienda1, tienda2, tienda3, tienda4]:
    tienda['Fecha de Compra'] = pd.to_datetime(tienda['Fecha de Compra'], format='%d/%m/%Y')

# Agrupar ventas por mes para ver tendencias temporales
ventas_mensuales = {}
for i, tienda in enumerate([tienda1, tienda2, tienda3, tienda4], 1):
    # Agrupar por año y mes
    ventas_mes = tienda.groupby(tienda['Fecha de Compra'].dt.to_period('M'))['Precio'].sum()
    ventas_mensuales[f'Tienda {i}'] = ventas_mes

# Información sobre tendencias temporales
print("\n=== TENDENCIAS TEMPORALES DE VENTAS ===")
print("-" * 50)
print("Las tendencias temporales podrían proporcionar información valiosa sobre:")
print("- Estacionalidad en las ventas")
print("- Meses de mayor y menor actividad")
print("- Impacto de campañas promocionales")
print("- Comparación de crecimiento entre tiendas")

# 3. Recomendaciones para mejorar las tiendas restantes
print("\n=== RECOMENDACIONES PARA MEJORAR LAS TIENDAS RESTANTES ===")
print("-" * 50)
print("""
Para maximizar el valor de las tiendas restantes, el Sr. Juan podría considerar:

1. MEJORA DE CALIFICACIONES:
   - Implementar un sistema de seguimiento de satisfacción del cliente
   - Capacitar al personal en atención al cliente
   - Mejorar la calidad de los productos en categorías con bajas calificaciones

2. OPTIMIZACIÓN DE ENVÍOS:
   - Analizar y renegociar contratos con empresas de logística
   - Implementar envío gratuito para compras superiores a cierto monto
   - Mejorar la eficiencia de embalaje y rutas de entrega

3. OPTIMIZACIÓN DE INVENTARIO:
   - Aumentar stock de productos más vendidos y reducir los menos populares
   - Enfocarse en categorías de mayor rentabilidad
   - Implementar promociones para productos de baja rotación

4. ESTRATEGIAS DE MARKETING:
   - Desarrollar campañas específicas para categorías rentables
   - Crear programas de fidelización para clientes recurrentes
   - Utilizar análisis geográfico para marketing localizado
""")

# Conclusión final
print("\n" + "=" * 80)
print("CONCLUSIÓN FINAL")
print("=" * 80)

print(f"""
El análisis de datos realizado para las tiendas de Alura Store ha proporcionado
una visión clara y objetiva del desempeño de cada establecimiento. Basándonos
en múltiples métricas y visualizaciones, nuestra recomendación es que el Sr. Juan
proceda con la venta de la {tienda_menos_eficiente}.

Esta decisión estratégica le permitirá obtener el capital necesario para su nuevo
emprendimiento mientras mantiene las unidades más rentables de su cadena actual.
Además, hemos proporcionado recomendaciones adicionales para mejorar el desempeño
de las tiendas restantes, maximizando así el valor total de su negocio.

Este análisis demuestra el poder de los datos para informar decisiones empresariales
importantes y proporciona un marco analítico que el Sr. Juan puede seguir utilizando
para evaluar el desempeño futuro de sus negocios.
""")

